In [1]:
import os
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import math
import json

# Constants

In [4]:
BONE_CONNECTIONS = [

    # Face - Neck - Shoulder
    [2,0,3],
    [2,0,4],

    # Elbow - Shoulder - Shoulder
    [7,3,4],
    [8,4,3],

    #elbow - shoulder - hip
    [7,3,5],
    [8,4,6],

    #wrist - elbow - shoulder
    [11,7,3],
    [12,8,4],

    #shoulder - hip - knee
    [3,5,9],
    [4,6,10],

    #hip - knee - ankle
    [5,9,13],
    [6,10,14],
]

# Now the processing

In [2]:
DRIVE_DIR = "H:/JHMDB"

JOINT_POSITION_FOLDER = f"{DRIVE_DIR}/joint_positions/joint_positions"
CLASSES = os.listdir(JOINT_POSITION_FOLDER)
#remove some random folders
for c in CLASSES: 
    if c[0] == ".": CLASSES.remove(c)

print(CLASSES)

['clap', 'swing_baseball', 'pour', 'run', 'wave', 'shoot_gun', 'shoot_ball', 'throw', 'golf', 'brush_hair', 'shoot_bow', 'stand', 'sit', 'climb_stairs', 'jump', 'pullup', 'pick', 'kick_ball', 'walk', 'push', 'catch']


In [7]:
for c in CLASSES:

    print(c)

    if not os.path.exists(f"{DRIVE_DIR}/processed_joints/{c}"):
        os.mkdir(f"{DRIVE_DIR}/processed_joints/{c}")

    for i in os.listdir(f"{JOINT_POSITION_FOLDER}/{c}"):
        
        if i[0] == ".": continue

        joint_positions = scipy.io.loadmat(f"{JOINT_POSITION_FOLDER}/{c}/{i}/joint_positions.mat")['pos_img']
        
        # uncomment this to show some bone plots

        # frame0 = f"{DRIVE_DIR}/Rename_Images/Rename_Images/{c}/{i}/00001.png"
        # plt.imshow(plt.imread(frame0))
        # plt.show()
        # for c in BONE_CONNECTIONS:
        #     # plt.figure(figsize=(15,15))
        #     plt.imshow(plt.imread(frame0))
        #     print(c)
        #     for x in c:
        #         plt.scatter(joint_positions[:,:,0][0][x], joint_positions[:,:,0][1][x])
        #     plt.show()

        bone_angles = []
        for j in range(joint_positions.shape[2]):

            frame_joint_positions = joint_positions[:,:,j]

            # frame_joint_positions[1] = frame_joint_positions[1] * -1

            one_frame_bone_angles = []
            for bone_connection in BONE_CONNECTIONS:
                
                j1 = frame_joint_positions[:,bone_connection[0]]
                j2 = frame_joint_positions[:,bone_connection[1]]
                j3 = frame_joint_positions[:,bone_connection[2]]

                bone_vector_1 = [j1[0] - j2[0], (j1[1] - j2[1])*-1]
                bone_vector_2 = [j3[0] - j2[0], (j3[1] - j2[1])*-1]

                if np.linalg.norm(bone_vector_1) == 0.0 or np.linalg.norm(bone_vector_2) == 0.0:
                    one_frame_bone_angles.append(None)
                    print("bone vector was on origin")
                    continue

                bone_unit_vector_1 = bone_vector_1 / np.linalg.norm(bone_vector_1)
                bone_unit_vector_2 = bone_vector_2 / np.linalg.norm(bone_vector_2)

                bone_angle = math.degrees(math.atan2(bone_unit_vector_1[1], bone_unit_vector_1[0]) - math.atan2(bone_unit_vector_2[1], bone_unit_vector_2[0]))

                one_frame_bone_angles.append(bone_angle)

                # plt.xlim(0,350)
                # plt.ylim(256,0)
                # plt.grid()
                # plt.plot([j2[0], j1[0]], [j2[1], j1[1]])
                # plt.plot([j2[0], j3[0]], [j2[1], j3[1]])

            bone_angles.append(one_frame_bone_angles)

        bone_angles = np.asarray(bone_angles)

        bone_angle_changes = []

        for f in range(bone_angles.shape[1])[2:]:

            one_bone_angle_changes = []
            for j in range(1, len(bone_angles[:,f])):

                angle_changes = []

                for k in range(1,39):

                    if k > j:
                        angle_changes.append(0)
                        continue

                    angle_changes.append(bone_angles[j,f] - bone_angles[j-k,f])

                one_bone_angle_changes.append(angle_changes)

            bone_angle_changes.append(one_bone_angle_changes)

        with open(f"{DRIVE_DIR}/processed_joints/{c}/{i}.json", 'w') as f:
            json.dump(bone_angle_changes, f)


clap
swing_baseball
pour
run
wave
shoot_gun
shoot_ball
throw
golf
brush_hair
shoot_bow
stand
sit
climb_stairs
jump
pullup
pick
kick_ball
walk
push
catch
